# 1. 模型

```
模型 (nn.Module)
|--模型创建
|   |--构建网络层
|   |   |--卷积层、池化层、激活函数层等
|   |--拼接网络层
|   |   |--LeNet、AlexNet、ResNet等
|--模型初始化
|   |--Xavier、Kaiming、均匀分布、正态分布等
```

## 1.1 模型创建 (以 `LeNet` 为例)

### 1). 构建子模块 `__init__()`
```python
def __init__(self, classes):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, classes)
```

### 2). 拼接子模块 `forward()`
```python
def forward(self, x):
    out = F.relu(self.conv1(x))
    out = F.max_pool2d(out, 2)

    out = F.relu(self.conv2(x))
    out = F.max_pool2d(out, 2)

    out = out.view(out.size(0), -1)
    
    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(out)
    
    return out
```

### 综上，构建 `LeNet` 的整个过程：
```python
class LeNet(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__() ## super 实现父类函数的调用，这里指的是 LeNet 调用父类 nn.Module 的初始化函数。
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)

        out = F.relu(self.conv2(x))
        out = F.max_pool2d(out, 2)

        out = out.view(out.size(0), -1)
        
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out
```

## 1.2 模型初始化

# 2. `nn.Module` 的详解

## 2.1 `torch.nn`

### 1). `nn.Module`
> 所有网络层的基类，管理网络属性。
>
> 关于 `nn.Module` 的总结：
> &emsp;&emsp; 1. 一个 `module` 可以包含多个子module；
>
> &emsp;&emsp; 2. 一个 `module` 相当于一个运算，必须实现 `forward()` 函数；
>
> &emsp;&emsp; 3. 每个 `module` 都有8个有序字典管理它的属性，重点关注 `modules` 和 `parameters`，管理模型和可学习参数；

**重点关注：**

**`parameters`:** `self._parameters = OrderDict()`，存储管理 `nn.Parameters` 类；

**`modules`：**`self._modules = OrderDict()`，存储管理 `nn.Modules` 类；

**`buffers`:** `self._buffers = OrderDict()`，存储管理缓冲属性，比如 BN层 中的均值、方差等；


### 2). `nn.Parameter`
> 张量(Tensor)子类，表示可学习参数，如 weight, bias。

### 3). `nn.functional`
函数具体实现，比如卷积、池化、激活函数等。

### 4). `nn.init`
参数初始化方法。

# 3. 模型容器 (Containers) 以及 `AlexNet` 的构建
```
模型容器 (Containers)
|--nn.Sequential
|   |-- 按顺序 包装多个网络层。
|   |-- 顺序性，各网络层之间严格按照顺序执行，常用语 block 构建。(整体上感觉 nn.Sequential 这个模型容器使用更方便)
|--nn.MuduleList
|   |-- 像 Python 的 list 一样包装多个网络层。
|   |-- 迭代性，常用于大量重复网络构建。
|--nn.ModuleDict
|   |-- 像 Python 的 dict 一样包装多个网络层。
|   |-- 索引性，常用语可选择的网络层。
```

## 3.1 Containers:

### 1). `nn.Sequential`

> 是 `nn.Module` 的容器，用于按顺序包装一组网络层。
>
> `nn.Sequential` 的特性：
> 
> &emsp;&emsp; 1. 顺序性：各网络层之间按照严格的顺序进行构建。
>
> &emsp;&emsp; 2. 自带 `forward()`：在自带的`forward()`中，通过for循环一次执行前向传播。


**示例：使用 `nn.Sequential` 构建 `LeNet`**

```python
## 方法1（网络层的名称使用默认的序号）;
class LeNetSequential(nn.Module):
    def __init__(self, classes):
        super(LeNetSequential, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(6, 16, 5),
            nn.ReLU()
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size[0], -1)
        x = self.classifier(x)
        return x




## 方法2（网络层的名称进行手动指定）
class LeNetSequentialOrderDict(nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()
        
        self.features = nn.Sequential(OrderDict({
            "conv1": nn.Conv2d(3, 6, 5),
            "relu1": nn.ReLU(inplace=True),
            "pool1": nn.MaxPool2d(kernel_size=2, stride=2),

            "conv2": nn.Conv2d(6, 16, 5),
            "relu2": nn.ReLU(inplace=True),
            "pool2": nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = nn.Sequential(OrderDict({
            "fc1": nn.Linear(16*5*5, 120),
            "relu3": nn.ReLU(inplace=True),
            "fc2": nn.Linear(120, 84),
            "relu4": nn.ReLU(inplace=True),
            "fc3": nn.Linear(84, classes),
        }))
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size[0], -1)
        x = self.classifier(x)
        return x
```


### 2). `nn.ModuleList`
> 也是 `nn.Module` 的容器，用于包装一组网络层，并以迭代的方式调用网络层。
> 
> 主要方法：
> 
> `append()`：在 ModuleList 后面添加网络层；
>
> `extend()`: 拼接两个 ModuleList；
>
> `insert()`: 指定在 ModuleList 中特定位置插入网络层；


**示例：用 `nn.ModuleList` 实现20个全连接层（每层有10个神经元）构成的网络**

```python
class FCModuleList(nn.Module):
    def __init__(self):
        super(FCModuleList, self).__init__()
        
        self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(20)]) ## 用列表生成式进行构造

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x

```

### 3. `nn.ModuleDict`
> 也是 `nn.Module` 的容器，用于包装一组网络层，以索引的方式调用网络层。
>
> 主要方法：
>
> `clear()`: 清空 ModuleDict
>
> `items()`: 返回可迭代的 key-value
>
> `keys()`: 返回字典的 key
>
> `values()`: 返回字典的 value
>
> `pop()`: 返回一对 key-value，并从字典中删除


**示例: Conv + Relu 的组合**
```python
class ModuleDict(nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        
        self.choices = nn.ModuleDict({
            "conv": nn.Conv2d(10, 10, 3),
            "pool": nn.MaxPool2d(3)
        })

        self.activations = nn.ModuleDict({
            "relu": nn.ReLU(),
            "prelu": nn.PReLU()
        })

    def forward(self, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x
```

## 3.2 `AlexNet`的构建
> `AlexNet`的特点：
>
> 1. 采用 ReLU 代替饱和激活函数(比如 Sigmoid)，减轻梯度消失；
>
> 2. 采用 LRN (Local Response Normalization)，对数据归一化，减轻梯度消失；
>
> 3. 采用 Dropout 提高全连接层的鲁棒性，增强网络的泛化能力；
>
> 4. 采用 Data Augmentation；

**`AlexNet`的具体实现**
 
```python
## 来自于 Pytorch 官方提供的 AlexNet 实现 (torchvision.models.AlexNet())
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        _log_api_usage_once(self)
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
```
